# Loan Outcome Prediction at the Time of Application

## Import Packages

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import re

## Load Account Data

In [2]:
train_flag = pd.read_csv('data/train/train_flag.csv')
train_flag.sample(5)

,uid,NAME_CONTRACT_TYPE,TARGET
216359,PIL21780442,Cash loans,0
236439,RYU09536562,Cash loans,0
244789,LQU54676787,Cash loans,0
154738,KSA70563039,Revolving loans,0
255391,NYT72241768,Cash loans,0


In [260]:
accounts_data = pd.read_json('data/train/accounts_data_train.json', typ='series', orient='records')
accounts_data[8]

[{'credit_type': 'Consumer credit',
  'loan_amount': 110565.0,
  'amount_overdue': 0.0,
  'open_date': '2017-11-05',
  'closed_date': '2018-09-08',
  'payment_hist_string': '000000000000000000000000000000',
  'uid': 'AAB09356880'},
 {'credit_type': 'Consumer credit',
  'loan_amount': 116280.0,
  'amount_overdue': 0.0,
  'open_date': '2018-06-26',
  'closed_date': '2019-05-28',
  'payment_hist_string': '000000000000000000000000000000000',
  'uid': 'AAB09356880'}]

## Extract Account Features

In [229]:
def accounts_features(account_info):
    account_info = sorted(account_info, key=lambda loan: datetime.strptime(loan['open_date'], '%Y-%m-%d'))
    loan_amount, amount_overdue, tenure = [], [], []
    late_days, late_months = 0, 0
    running_loans, settled_loans = 0, 0
    is_latest_loan_settled = 0
    
    for loan in account_info:
        loan_amount.append(loan['loan_amount'])
        amount_overdue.append(loan['amount_overdue'])
        processed_hist_string = re.findall('...', loan['payment_hist_string'])
        late_days += sum([int(x) for x in processed_hist_string])
        late_months += sum(int(x) > 0 for x in processed_hist_string)
        if loan['closed_date'] is not None:
            closed_date = datetime.strptime(loan['closed_date'], '%Y-%m-%d')
            open_date = datetime.strptime(loan['open_date'], '%Y-%m-%d')
            tenure.append(closed_date - open_date)
            
            settled_loans += 1
        else:
            running_loans += 1
    
    if account_info[-1]['closed_date'] is not None:
        is_latest_loan_settled = 1
    
    first_loan_date = datetime.strptime(account_info[0]['open_date'], '%Y-%m-%d')
    last_loan_date = datetime.strptime(account_info[-1]['open_date'], '%Y-%m-%d')
    period = last_loan_date - first_loan_date
    
    loan_amount = [x for x in loan_amount if x is not None]
    amount_overdue = [x for x in amount_overdue if x is not None]
    tenure = [x for x in tenure if x is not None]
    features = {
        'uid': account_info[0]['uid'],
        'n_loans': len(account_info),
        
        # loan amount features
        'total_loan_amount': sum(loan_amount),
        'mean_loan_amount': sum(loan_amount) / len(account_info),
        'min_loan_amount': min(loan_amount) if len(loan_amount) > 0 else 0,
        'max_loan_amount': max(loan_amount) if len(loan_amount) > 0 else 0,
        
        # amount overdue features
        'amount_overdue': sum(amount_overdue),
        'overdue_ratio': sum(amount_overdue) / sum(loan_amount) if sum(loan_amount) > 0 else 0,
        
        # loan tenure features
        'total_tenure': sum([t.days for t in tenure]),
        'mean_tenure': sum([t.days for t in tenure]) / len(account_info),
        'min_tenure': min([t.days for t in tenure]) if len(tenure) > 0 else 0,
        'max_tenure': max([t.days for t in tenure]) if len(tenure) > 0 else 0,
        'days_per_rupee': sum([t.days for t in tenure]) / sum(loan_amount) if sum(loan_amount) > 0 else 0,
        
        # loan settlement features
        'running_loans': running_loans,
        'settled_loans': settled_loans,
        'settlement_ratio': settled_loans / (settled_loans + running_loans),
        'is_latest_loan_settled': is_latest_loan_settled,
        
        # late payment features
        'late_days': late_days,
        'late_months': late_months,
        
        # date features
        'loan_frequency': len(account_info) / period.days if period.days > 0 else len(account_info) / 1000,
        'days_since_last_loan': 0
    }
    return features

In [230]:
[x for x in processed_hist_string]

['000', '010', '000']

In [231]:
late_days, late_months = 0, 0
processed_hist_string = re.findall('...', '000010000')
late_days += sum([int(x) for x in processed_hist_string])
late_months += sum([int(x) > 0 for x in processed_hist_string])

In [223]:
accounts_features(accounts_data[120])

{'uid': 'AAJ18982277',
 'n_loans': 3,
 'total_loan_amount': 3427380.0,
 'mean_loan_amount': 1142460.0,
 'min_loan_amount': 0.0,
 'max_loan_amount': 3359880.0,
 'amount_overdue': 0.0,
 'overdue_ratio': 0.0,
 'total_tenure': 341,
 'mean_tenure': 113.66666666666667,
 'min_tenure': 341,
 'max_tenure': 341,
 'days_per_rupee': 9.949290711855703e-05,
 'running_loans': 2,
 'settled_loans': 1,
 'settlement_ratio': 0.3333333333333333,
 'is_latest_loan_settled': 0,
 'late_days': 22,
 'late_months': 1,
 'loan_frequency': 0.010714285714285714,
 'days_since_last_loan': 0}

## Load Enquiries Data

In [224]:
enquiry_data = pd.read_json('data/train/enquiry_data_train.json', typ='series', orient='records')
enquiry_data[10]

[{'enquiry_type': 'Real estate loan',
  'enquiry_amt': 90000,
  'enquiry_date': '2020-12-06',
  'uid': 'AAB09356880'},
 {'enquiry_type': 'Cash loans',
  'enquiry_amt': 67000,
  'enquiry_date': '2020-10-10',
  'uid': 'AAB09356880'}]

In [225]:
for data in accounts_data:
    if data[0]['uid'] == 'AAB09356880':
        #print(data)
        break
data

[{'credit_type': 'Consumer credit',
  'loan_amount': 110565.0,
  'amount_overdue': 0.0,
  'open_date': '2017-11-05',
  'closed_date': '2018-09-08',
  'payment_hist_string': '000000000000000000000000000000',
  'uid': 'AAB09356880'},
 {'credit_type': 'Consumer credit',
  'loan_amount': 116280.0,
  'amount_overdue': 0.0,
  'open_date': '2018-06-26',
  'closed_date': '2019-05-28',
  'payment_hist_string': '000000000000000000000000000000000',
  'uid': 'AAB09356880'}]

In [236]:
def enquiries_features(enquiry_info):
    enquiry_info = sorted(enquiry_info, key=lambda enquiry: datetime.strptime(enquiry['enquiry_date'], '%Y-%m-%d'))
    
    enquiry_amount = []
    for enquiry in enquiry_info:
        enquiry_amount.append(enquiry['enquiry_amt'])
    
    first_enq_date = datetime.strptime(enquiry_info[0]['enquiry_date'], '%Y-%m-%d')
    last_enq_date = datetime.strptime(enquiry_info[-1]['enquiry_date'], '%Y-%m-%d')
    period = last_enq_date - first_enq_date
        
    features = {
        'uid': enquiry_info[0]['uid'],
        'n_enquiries': len(enquiry_info),
        'total_enquiry_amount': sum(enquiry_amount),
        'mean_enquiry_amount': sum(enquiry_amount) / len(enquiry_info),
        'min_enquiry_amount': min(enquiry_amount),
        'max_enquiry_amount': max(enquiry_amount),
        'enquiry_frequency': len(enquiry_info) / period.days if period.days > 0 else len(enquiry_info) / 1000
    }
    
    return features

In [237]:
enquiries_features(enquiry_data[0])

{'uid': 'AAA08065248',
 'n_enquiries': 11,
 'total_enquiry_amount': 2064658,
 'mean_enquiry_amount': 187696.18181818182,
 'min_enquiry_amount': 36082,
 'max_enquiry_amount': 364751,
 'enquiry_frequency': 0.016417910447761194}

## Create Train DataFrame

In [238]:
acc_data = []
for account_info in accounts_data:
    acc_data.append(accounts_features(account_info))

In [245]:
acc_df = pd.DataFrame(acc_data)
acc_df.head()

,uid,n_loans,total_loan_amount,mean_loan_amount,min_loan_amount,max_loan_amount,amount_overdue,overdue_ratio,total_tenure,mean_tenure,...,max_tenure,days_per_rupee,running_loans,settled_loans,settlement_ratio,is_latest_loan_settled,late_days,late_months,loan_frequency,days_since_last_loan
0,AAA09044550,2,277245.000,138622.500,4500.000,272745.000,0.0,0.0,1022,511.000,...,518,0.003686,0,2,1.000,1,68,3,0.010101,0
1,AAA10545297,1,80996.445,80996.445,80996.445,80996.445,0.0,0.0,0,0.000,...,0,0.000000,1,0,0.000,0,0,0,0.001000,0
2,AAA14112888,1,43771.500,43771.500,43771.500,43771.500,0.0,0.0,92,92.000,...,92,0.002102,0,1,1.000,1,0,0,0.001000,0
3,AAA20326915,8,591597.000,73949.625,0.000,235800.000,0.0,0.0,1937,242.125,...,1450,0.003274,3,5,0.625,0,14477,31,0.003777,0
4,AAA31604840,5,1591960.500,318392.100,41845.500,687150.000,0.0,0.0,927,185.400,...,549,0.000582,3,2,0.400,0,0,0,0.003751,0


In [240]:
enq_data = []
for enq_info in enquiry_data:
    enq_data.append(enquiries_features(enq_info))

In [246]:
enq_df = pd.DataFrame(enq_data)
enq_df.head()

,uid,n_enquiries,total_enquiry_amount,mean_enquiry_amount,min_enquiry_amount,max_enquiry_amount,enquiry_frequency
0,AAA08065248,11,2064658,187696.181818,36082,364751,0.016418
1,AAA09044550,26,2659000,102269.230769,5000,197000,0.022241
2,AAA10545297,14,1317000,94071.428571,5000,192000,0.016548
3,AAA14112888,15,1465000,97666.666667,17000,185000,0.012723
4,AAA20326915,1,66000,66000.000000,66000,66000,0.001000


In [252]:
train_flag.uid = train_flag.uid.astype(str)
acc_df.uid = acc_df.uid.astype(str)
enq_df.uid = enq_df.uid.astype(str)

In [257]:
acc_df.uid.astype(str).dtype

dtype('O')

In [259]:
train_df = pd.merge(pd.merge(train_flag, acc_df, on='uid'), enq_df, on='uid')
train_df.sample(5)

,uid,NAME_CONTRACT_TYPE,TARGET,n_loans,total_loan_amount,mean_loan_amount,min_loan_amount,max_loan_amount,amount_overdue,overdue_ratio,...,late_days,late_months,loan_frequency,days_since_last_loan,n_enquiries,total_enquiry_amount,mean_enquiry_amount,min_enquiry_amount,max_enquiry_amount,enquiry_frequency
132976,MCZ66699917,Cash loans,0,2,1442700.00,721350.00,92700.00,1350000.0,0.0,0.0,...,0,0,0.004640,0,13,1230000,94615.384615,5000,197000,0.025145
41407,KJF07258735,Revolving loans,0,3,1710000.00,570000.00,135000.00,1350000.0,0.0,0.0,...,0,0,0.010169,0,5,1183066,236613.200000,62113,471300,0.021186
152124,AKP48023040,Cash loans,0,9,2165553.90,240617.10,13500.00,1080000.0,0.0,0.0,...,0,0,0.005573,0,4,416000,104000.000000,54000,140000,0.005076
166407,ASE87313818,Cash loans,0,9,681348.78,75705.42,20173.14,153000.0,0.0,0.0,...,0,0,0.004747,0,19,2124000,111789.473684,15000,196000,0.012829
5639,SKE26627541,Cash loans,0,3,1167300.00,389100.00,157500.00,739800.0,0.0,0.0,...,15,1,0.006494,0,24,2800000,116666.666667,23000,194000,0.031088


In [261]:
train_df.to_csv('train.csv', index=False)

## Create Test DataFrame

In [262]:
test_flag = pd.read_csv('data/test/test_flag.csv')
test_flag.sample(5)

,uid,NAME_CONTRACT_TYPE
1142,WHJ66285434,Cash loans
43531,CAR94353830,Cash loans
2068,NPS06524156,Cash loans
10327,ELG18486477,Cash loans
14233,OLF09106375,Cash loans


In [264]:
accounts_data = pd.read_json('data/test/accounts_data_test.json', typ='series', orient='records')
accounts_data[0]

[{'credit_type': 'Consumer credit',
  'loan_amount': 31630.5,
  'amount_overdue': 0.0,
  'open_date': '2014-03-30',
  'closed_date': '2014-11-29',
  'payment_hist_string': '000000000000000000000000',
  'uid': 'AAA14437029'},
 {'credit_type': 'Consumer credit',
  'loan_amount': 14613.39,
  'amount_overdue': 0.0,
  'open_date': '2014-06-01',
  'closed_date': '2014-11-03',
  'payment_hist_string': '000000000000000',
  'uid': 'AAA14437029'},
 {'credit_type': 'Credit card',
  'loan_amount': 54000.0,
  'amount_overdue': 0.0,
  'open_date': '2015-12-13',
  'closed_date': '2019-09-21',
  'payment_hist_string': '000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
  'uid': 'AAA14437029'},
 {'credit_type': 'Consumer credit',
  'loan_amount': 27076.5,
  'amount_overdue': 0.0,
  'open_date': '2015-11-11',
  'closed_date': '2016-11-24',
  'payment_hist_string': '000000000000000000000000000000000000',
  'uid': 'AAA1

In [265]:
enquiry_data = pd.read_json('data/test/enquiry_data_test.json', typ='series', orient='records')
enquiry_data[10]

[{'enquiry_type': 'Interbank credit',
  'enquiry_amt': 176000,
  'enquiry_date': '2020-11-01',
  'uid': 'AAE38809797'},
 {'enquiry_type': 'Cash loans',
  'enquiry_amt': 136000,
  'enquiry_date': '2020-08-22',
  'uid': 'AAE38809797'},
 {'enquiry_type': 'Loan for purchase of shares (margin lending)',
  'enquiry_amt': 152000,
  'enquiry_date': '2020-12-21',
  'uid': 'AAE38809797'},
 {'enquiry_type': 'Cash loans',
  'enquiry_amt': 70000,
  'enquiry_date': '2020-11-06',
  'uid': 'AAE38809797'},
 {'enquiry_type': 'Real estate loan',
  'enquiry_amt': 9000,
  'enquiry_date': '2020-03-20',
  'uid': 'AAE38809797'},
 {'enquiry_type': 'Mortgage',
  'enquiry_amt': 177000,
  'enquiry_date': '2020-01-14',
  'uid': 'AAE38809797'},
 {'enquiry_type': 'Consumer credit',
  'enquiry_amt': 27000,
  'enquiry_date': '2020-07-28',
  'uid': 'AAE38809797'}]

In [266]:
acc_data = []
for account_info in accounts_data:
    acc_data.append(accounts_features(account_info))

In [267]:
acc_df = pd.DataFrame(acc_data)
acc_df.head()

,uid,n_loans,total_loan_amount,mean_loan_amount,min_loan_amount,max_loan_amount,amount_overdue,overdue_ratio,total_tenure,mean_tenure,...,max_tenure,days_per_rupee,running_loans,settled_loans,settlement_ratio,is_latest_loan_settled,late_days,late_months,loan_frequency,days_since_last_loan
0,AAA14437029,12,3736315.890,311359.657500,14613.39,1575000.0,0.0,0.0,5220,435.000000,...,1378,0.001397,2,10,0.833333,0,1500,10,0.005894,0
1,AAB12915377,3,408007.980,136002.660000,45532.98,247500.0,0.0,0.0,182,60.666667,...,182,0.000446,2,1,0.333333,0,42,2,0.002865,0
2,AAB55088883,6,327394.485,54565.747500,4500.00,114471.9,0.0,0.0,921,153.500000,...,306,0.002813,2,4,0.666667,0,0,0,0.002896,0
3,AAB68152393,7,1806736.500,258105.214286,63049.50,711000.0,0.0,0.0,2686,383.714286,...,987,0.001487,2,5,0.714286,1,984,8,0.003453,0
4,AAC29580834,5,2867536.800,573507.360000,40860.00,1422000.0,0.0,0.0,812,162.400000,...,446,0.000283,3,2,0.400000,0,0,0,0.002992,0


In [268]:
enq_data = []
for enq_info in enquiry_data:
    enq_data.append(enquiries_features(enq_info))

In [269]:
enq_df = pd.DataFrame(enq_data)
enq_df.head()

,uid,n_enquiries,total_enquiry_amount,mean_enquiry_amount,min_enquiry_amount,max_enquiry_amount,enquiry_frequency
0,AAA02107680,1,143000,143000.000000,143000,143000,0.001000
1,AAA14437029,4,369000,92250.000000,12000,174000,0.007707
2,AAB12915377,1,137000,137000.000000,137000,137000,0.001000
3,AAB55088883,14,1105000,78928.571429,6000,176000,0.018373
4,AAB68152393,17,1987000,116882.352941,9000,186000,0.017472


In [270]:
test_df = pd.merge(pd.merge(test_flag, acc_df, on='uid'), enq_df, on='uid')
test_df.sample(5)

,uid,NAME_CONTRACT_TYPE,n_loans,total_loan_amount,mean_loan_amount,min_loan_amount,max_loan_amount,amount_overdue,overdue_ratio,total_tenure,...,late_days,late_months,loan_frequency,days_since_last_loan,n_enquiries,total_enquiry_amount,mean_enquiry_amount,min_enquiry_amount,max_enquiry_amount,enquiry_frequency
11889,TLW48482098,Cash loans,3,740398.50,246799.50,208498.50,306900.0,0.0,0.0,731,...,0,0,0.002179,0,5,553000,110600.000000,61000,176000,0.005995
30454,PBC49308103,Cash loans,6,935680.50,155946.75,67176.00,225000.0,0.0,0.0,1289,...,0,0,0.004370,0,1,134000,134000.000000,134000,134000,0.001000
20799,IJM48794397,Cash loans,3,262575.00,87525.00,29817.00,135000.0,0.0,0.0,486,...,0,0,0.003337,0,5,604000,120800.000000,49000,182000,0.006046
8452,VXE18458443,Cash loans,3,1198215.00,399405.00,126000.00,671715.0,0.0,0.0,3567,...,0,0,0.001278,0,12,1510000,125833.333333,8000,177000,0.017647
22967,UZX69165329,Cash loans,6,4842603.36,807100.56,45234.36,3150000.0,0.0,0.0,4027,...,0,0,0.002976,0,26,2339000,89961.538462,11000,183000,0.033333


In [271]:
test_df.to_csv('test.csv', index=False)